# Getting started with PyProBE

In [29]:
%%capture
%pip install matplotlib

In [30]:
from pprint import pprint

import pyprobe

%matplotlib inline

## Convert data to standard format

Create the cell object and load some data. If this is the first time that the data has been loaded, it must first be converted into the standard format for PyProBE. The `import_from_cycler` method will then add the data directly to the `procedure` dictionary of the cell with the given `procedure_name` as its key.

In [31]:
# Describe the cell. Required fields are 'Name'.
info_dictionary = {
    "Name": "Sample cell",
    "Chemistry": "NMC622",
    "Nominal Capacity [Ah]": 0.04,
    "Cycler number": 1,
    "Channel number": 1,
}

# Create a cell object
cell = pyprobe.Cell(info=info_dictionary)

data_directory = "../../../tests/sample_data/neware"

cell.import_from_cycler(
    procedure_name="Sample",
    cycler="neware",
    input_data_path=data_directory + "/sample_data_neware.xlsx",
)

If a file named `README.yaml`, sits alongside your data, it will automatically be imported. You can also specify a custom path for this file. The README file contains descriptions of the experiments and steps in the procedure:

In [32]:
import yaml

with open(data_directory + "/README.yaml") as f:
    pprint(yaml.safe_load(f))

{'Break-in Cycles': {'Cycle': {'Count': 5, 'End': 7, 'Start': 4},
                     'Steps': {4: 'Discharge at 4 mA until 3 V',
                               5: 'Rest for 2 hours',
                               6: 'Charge at 4 mA until 4.2 V, Hold at 4.2 V '
                                  'until 0.04 A',
                               7: 'Rest for 2 hours'}},
 'Discharge Pulses': {'Cycle': {'Count': 10, 'End': 12, 'Start': 9},
                      'Steps': {9: 'Rest for 10 seconds',
                                10: 'Discharge at 20 mA for 0.2 hours or until '
                                    '3 V',
                                11: 'Rest for 30 minutes',
                                12: 'Rest for 1.5 hours'}},
 'Initial Charge': {'Steps': {1: 'Rest for 4 hours',
                              2: 'Charge at 4mA until 4.2 V, Hold at 4.2 V '
                                 'until 0.04 A',
                              3: 'Rest for 2 hours'}}}


Once loaded, these can be accessed through the `experiment_names` and `step_descriptions` attributes of the procedure:

In [33]:
print("Experiment names: ", cell.procedure["Sample"].experiment_names)
print("Step Descriptions: ")
pprint(cell.procedure["Sample"].step_descriptions)

Experiment names:  ['Initial Charge', 'Break-in Cycles', 'Discharge Pulses']
Step Descriptions: 
{'Description': ['Rest for 4 hours',
                 'Charge at 4mA until 4.2 V, Hold at 4.2 V until 0.04 A',
                 'Rest for 2 hours',
                 'Discharge at 4 mA until 3 V',
                 'Rest for 2 hours',
                 'Charge at 4 mA until 4.2 V, Hold at 4.2 V until 0.04 A',
                 'Rest for 2 hours',
                 'Rest for 10 seconds',
                 'Discharge at 20 mA for 0.2 hours or until 3 V',
                 'Rest for 30 minutes',
                 'Rest for 1.5 hours'],
 'Step': [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12]}


Alternatively, if you need to view data quickly and have not prepared a README file, the data will load without one (we will temporarily rename the README file to prevent it being automatically detected):

In [34]:
import os

os.rename(data_directory + "/README.yaml", data_directory + "/README_bak.yaml")
cell.import_from_cycler(
    procedure_name="Sample Quick",
    cycler="neware",
    input_data_path=data_directory + "/sample_data_neware.xlsx",
)
os.rename(data_directory + "/README_bak.yaml", data_directory + "/README.yaml")

This procedure will have empty `experiment_names` and `step_descriptions` attributes:

In [35]:
print("Experiment names: ", cell.procedure["Sample Quick"].experiment_names)
print("Step Descriptions: ")
pprint(cell.procedure["Sample Quick"].step_descriptions)

Experiment names:  []
Step Descriptions: 
{'Description': [], 'Step': []}


The dashboard can be launched as soon as procedures have been added to the cell (uncomment to run when outside docs environment):

In [36]:
# pyprobe.launch_dashboard([cell]) # noqa: ERA001

The raw data is accessible as a dataframe with the data property:

In [37]:
print(cell.procedure["Sample"].data)

shape: (789_589, 9)
┌────────────┬──────┬───────┬─────────────┬───┬─────────────┬────────────┬────────────┬────────────┐
│ Time [s]   ┆ Step ┆ Event ┆ Current [A] ┆ … ┆ Capacity    ┆ Date       ┆ Procedure  ┆ Procedure  │
│ ---        ┆ ---  ┆ ---   ┆ ---         ┆   ┆ [Ah]        ┆ ---        ┆ Time [s]   ┆ Capacity   │
│ f64        ┆ i64  ┆ i64   ┆ f64         ┆   ┆ ---         ┆ datetime[μ ┆ ---        ┆ [Ah]       │
│            ┆      ┆       ┆             ┆   ┆ f64         ┆ s]         ┆ f64        ┆ ---        │
│            ┆      ┆       ┆             ┆   ┆             ┆            ┆            ┆ f64        │
╞════════════╪══════╪═══════╪═════════════╪═══╪═════════════╪════════════╪════════════╪════════════╡
│ 0.0        ┆ 2    ┆ 0     ┆ 0.003999    ┆ … ┆ 0.04139     ┆ 2024-02-29 ┆ 0.0        ┆ 0.0        │
│            ┆      ┆       ┆             ┆   ┆             ┆ 09:20:29.0 ┆            ┆            │
│            ┆      ┆       ┆             ┆   ┆             ┆ 94       

Individual columns can be returned as 1D numpy arrays with the `get()` method:

In [38]:
current = (
    cell.procedure["Sample"].experiment("Break-in Cycles").charge(0).get("Current [A]")
)
print(type(current), current)

<class 'numpy.ndarray'> [0.00399931 0.00400001 0.00400004 ... 0.00040614 0.0004023  0.0004    ]


Multiple columns can be returned at once:

In [39]:
current, voltage = (
    cell.procedure["Sample"]
    .experiment("Break-in Cycles")
    .charge(0)
    .get("Current [A]", "Voltage [V]")
)
print("Current = ", current)
print("Voltage = ", voltage)

Current =  [0.00399931 0.00400001 0.00400004 ... 0.00040614 0.0004023  0.0004    ]
Voltage =  [3.2895 3.2962 3.2979 ... 4.2001 4.2001 4.2001]


A mistyped column will raise an error and suggest close matches if available:

In [42]:
current, voltage = (
    cell.procedure["Sample"]
    .experiment("Break-in Cycles")
    .charge(0)
    .get("Crrent [A]", "Voltge [V]")
)
print("Current [A] = ", current)
print("Voltage [V]= ", voltage)

10:24:40 | ERROR | pyprobe.result:get:365 - Column "Crrent [A]" not found. Did you mean "Current [A]"? | Context: {}
10:24:40 | ERROR | pyprobe.result:get:365 - Column "Voltge [V]" not found. Did you mean "Voltage [V]"? | Context: {}


ValueError: 
- Column "Crrent [A]" not found. Did you mean "Current [A]"?
- Column "Voltge [V]" not found. Did you mean "Voltage [V]"?

If the column is completely mistyped an error will be thrown and all available columns will be listed:

In [ ]:
voltage = (
    cell.procedure["Sample"].experiment("Break-in Cycles").charge(0).get("valoolashaka")
)
print(type(voltage), voltage)

# Cutoff ratio (similarity limit) of 0.5 means most vaguely similar names will be matched, but totally different names will not.

10:33:05 | ERROR | pyprobe.result:get:369 - Column "valoolashaka" not found and no close match found. Available columns: Step, Date, Event, Capacity [Ah], Voltage [V], Procedure Time [s], Current [A], Time [s], Procedure Capacity [Ah], Experiment Time [s], Experiment Capacity [Ah], Step Time [s], Step Capacity [Ah] | Context: {}


<class 'NoneType'> None


And different unit can be returned on command:

In [ ]:
current_mA = (
    cell.procedure["Sample"].experiment("Break-in Cycles").charge(0).get("Current [mA]")
)
print("Current [mA] = ", current_mA)

10:15:03 | ERROR | pyprobe.result:get:365 - Column "Current [mA]" not found. Did you mean "Current [A]"? | Context: {}


ValueError: 
- Column "Current [mA]" not found. Did you mean "Current [A]"?

Any part of the procedure can be plotted quickly using the ```plot``` method:

In [ ]:
cell.procedure["Sample"].experiment("Break-in Cycles").plot(
    x="Experiment Time [s]",
    y="Voltage [V]",
)

We can use the `analysis` to further analyse the data. For the `'Break-in Cycles'` we will use the `cycling` analysis module and the functions within. These functions return `Result` objects, so they can be interacted with in the same ways as raw data:

In [ ]:
from pyprobe.analysis import cycling

cycling_summary = cycling.summary(
    input_data=cell.procedure["Sample"].experiment("Break-in Cycles"),
)
print(type(cycling_summary))

print(cycling_summary.data)

And it can be plotted as normal too:

In [ ]:
cycling_summary.plot(x="Capacity Throughput [Ah]", y="Discharge Capacity [mAh]")

As the procedure that we imported without a README file does not contain experiment information, the `Break-in Cycles` will not work on it:

In [ ]:
cycling_summary = cycling.summary(
    input_data=cell.procedure["Sample Quick"].experiment("Break-in Cycles"),
)

However, all other filters will still work as expected.